In [47]:
import pytorch_lightning as pl
import pandas as pd
import torch
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, OrdinalEncoder, FunctionTransformer, MinMaxScaler
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import mean_absolute_error, mean_squared_error
from torchmetrics import MeanSquaredError
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping

In [2]:
df = pd.read_csv('data/train.csv')
dfm = pd.read_csv('data/meal_info.csv')
dfc = pd.read_csv('data/fulfilment_center_info.csv')
df = pd.merge(df, dfm, on='meal_id', how='left')
df = pd.merge(df, dfc, on='center_id', how='left')

In [3]:
df.head()

,id,week,center_id,meal_id,checkout_price,base_price,emailer_for_promotion,homepage_featured,num_orders,category,cuisine,city_code,region_code,center_type,op_area
0,1379560,1,55,1885,136.83,152.29,0,0,177,Beverages,Thai,647,56,TYPE_C,2.0
1,1466964,1,55,1993,136.83,135.83,0,0,270,Beverages,Thai,647,56,TYPE_C,2.0
2,1346989,1,55,2539,134.86,135.86,0,0,189,Beverages,Thai,647,56,TYPE_C,2.0
3,1338232,1,55,2139,339.50,437.53,0,0,54,Beverages,Indian,647,56,TYPE_C,2.0
4,1448490,1,55,2631,243.50,242.50,0,0,40,Beverages,Indian,647,56,TYPE_C,2.0


In [4]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
id,456548.0,1.250096e+06,144354.822378,1000000.00,1124998.75,1250183.50,1375140.25,1499999.00
week,456548.0,7.476877e+01,41.524956,1.00,39.00,76.00,111.00,145.00
center_id,456548.0,8.210580e+01,45.975046,10.00,43.00,76.00,110.00,186.00
meal_id,456548.0,2.024337e+03,547.420920,1062.00,1558.00,1993.00,2539.00,2956.00
checkout_price,456548.0,3.322389e+02,152.939723,2.97,228.95,296.82,445.23,866.27
base_price,456548.0,3.541566e+02,160.715914,55.35,243.50,310.46,458.87,866.27
emailer_for_promotion,456548.0,8.115247e-02,0.273069,0.00,0.00,0.00,0.00,1.00
homepage_featured,456548.0,1.091999e-01,0.311890,0.00,0.00,0.00,0.00,1.00
num_orders,456548.0,2.618728e+02,395.922798,13.00,54.00,136.00,324.00,24299.00
city_code,456548.0,6.015534e+02,66.195914,456.00,553.00,596.00,651.00,713.00


In [5]:
df["category"] = df["category"].astype("category")
columns_to_drop = ["num_orders", "id"]
test_start_week = df["week"].max() * 0.8
validation_start_week = df["week"].max() * 0.6
df_train = df[df["week"] < validation_start_week]
df_validation = df[(df["week"] >= validation_start_week) & (df["week"] < test_start_week)]
df_test = df[df["week"] >= test_start_week]
X_train = df_train.drop(columns=columns_to_drop)
y_train = df_train["num_orders"]
X_validation = df_validation.drop(columns=columns_to_drop)
y_validation = df_validation["num_orders"]
X_test = df_test.drop(columns=columns_to_drop)
y_test = df_test["num_orders"]

In [6]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Index: 263590 entries, 0 to 263589
Data columns (total 13 columns):
 #   Column                 Non-Null Count   Dtype   
---  ------                 --------------   -----   
 0   week                   263590 non-null  int64   
 1   center_id              263590 non-null  int64   
 2   meal_id                263590 non-null  int64   
 3   checkout_price         263590 non-null  float64 
 4   base_price             263590 non-null  float64 
 5   emailer_for_promotion  263590 non-null  int64   
 6   homepage_featured      263590 non-null  int64   
 7   category               263590 non-null  category
 8   cuisine                263590 non-null  object  
 9   city_code              263590 non-null  int64   
 10  region_code            263590 non-null  int64   
 11  center_type            263590 non-null  object  
 12  op_area                263590 non-null  float64 
dtypes: category(1), float64(3), int64(7), object(2)
memory usage: 26.4+ MB


In [7]:
X_train.describe().T

,count,mean,std,min,25%,50%,75%,max
week,263590.0,44.542365,24.750090,1.00,23.00,45.00,66.00,86.00
center_id,263590.0,82.191878,45.970215,10.00,43.00,76.00,110.00,186.00
meal_id,263590.0,2018.845719,546.804240,1062.00,1543.00,1971.00,2539.00,2956.00
checkout_price,263590.0,329.775977,152.226226,2.97,229.89,291.06,438.50,738.23
base_price,263590.0,350.881830,157.041646,55.35,243.50,309.43,455.93,738.23
emailer_for_promotion,263590.0,0.080318,0.271785,0.00,0.00,0.00,0.00,1.00
homepage_featured,263590.0,0.108984,0.311619,0.00,0.00,0.00,0.00,1.00
city_code,263590.0,601.524018,66.269349,456.00,553.00,596.00,651.00,713.00
region_code,263590.0,56.577393,17.627253,23.00,34.00,56.00,77.00,93.00
op_area,263590.0,4.082859,1.089724,0.90,3.60,4.00,4.50,7.00


In [8]:
X_train.nunique()

week                       86
center_id                  77
meal_id                    51
checkout_price           1892
base_price               1745
emailer_for_promotion       2
homepage_featured           2
category                   14
cuisine                     4
city_code                  51
region_code                 8
center_type                 3
op_area                    30
dtype: int64

In [9]:
# df = pd.DataFrame({
#     'Color': ['Red', 'Blue', 'Green', 'Blue', 'Red'],  # Categorical
#     'Size': ['S', 'M', 'L', 'S', 'L'],  # Ordinal
#     "Category": ['A', 'B', 'A', 'B', 'A'],  # Categorical
#     'Price': [10, 20, 15, 25, 30],  # Continuous
#     'Week': [10, 20, 15, 25, 30]  # Continuous
# })
# ordinal_columns = ["Category"]
# one_hot_columns = ["Size", "Color"]
# numerical_columns = ["Price"]
# unchanged_columns = ["Week"]

ordinal_columns = ["center_id", "meal_id", "category", "city_code", "region_code"]
one_hot_columns = ["cuisine", "center_type"]
numerical_columns = ["checkout_price", "base_price", "op_area"]
unchanged_columns = ["week", "emailer_for_promotion", "homepage_featured"]


def transform_data(df, pipeline):
    if not pipeline:
        ordinal_encoder_pipeline = Pipeline(
            steps=[
                ("ordinal_encoder", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)),
                ("functional_transformer", FunctionTransformer(lambda df: df + 1)),
            ]
        )
        one_hot_encoder_pipeline = Pipeline(
            steps=[
                ("one_hot_encoder", OneHotEncoder(handle_unknown="ignore", sparse_output=False))
            ]
        )
        numerical_encoder_pipeline = Pipeline(
            steps=[
                ("simple_imputer", SimpleImputer(strategy='constant', fill_value=0)),
                ("min_max_scaler", MinMaxScaler())
            ]
        )
        preprocessor = ColumnTransformer(
            transformers=[
                ('ordinal_encoder_pipeline', ordinal_encoder_pipeline, ordinal_columns),
                ("one_hot_encoder_pipeline", one_hot_encoder_pipeline, one_hot_columns),
                ("numerical_encoder_pipeline", numerical_encoder_pipeline, numerical_columns)
            ], remainder='passthrough')
        pipeline = Pipeline(steps=[("preprocessor", preprocessor)])
        encoded = pipeline.fit_transform(df)
    else:
        encoded = pipeline.transform(df)
    new_one_hot_columns = list(
        pipeline.named_steps["preprocessor"].named_transformers_["one_hot_encoder_pipeline"].get_feature_names_out(
            one_hot_columns))
    new_column_names = ordinal_columns + new_one_hot_columns + numerical_columns + unchanged_columns
    return pd.DataFrame(encoded, columns=new_column_names), pipeline


X_train, pipeline = transform_data(X_train, None)

In [10]:
X_test, pipeline = transform_data(X_test, pipeline)
X_validation, pipeline = transform_data(X_validation, pipeline)

In [11]:
X_train[ordinal_columns] = X_train[ordinal_columns].astype("int")
X_test[ordinal_columns] = X_test[ordinal_columns].astype("int")
X_validation[ordinal_columns] = X_validation[ordinal_columns].astype("int")
categorical_cardinalities = [X_train[column].max() for column in ordinal_columns]
X_train_numeric = X_train.drop(columns=ordinal_columns)
X_train_embedding = X_train[ordinal_columns]

X_test_numeric = X_test.drop(columns=ordinal_columns)
X_test_embedding = X_test[ordinal_columns]
X_validation_numeric = X_validation.drop(columns=ordinal_columns)
X_validation_embedding = X_validation[ordinal_columns]

In [12]:
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(device)


class CustomDataset(Dataset):
    def __init__(self, X_numeric, X_embedding, y):
        self.X_numeric = torch.tensor(X_numeric.values, dtype=torch.float32).to(device)
        self.X_embedding = torch.tensor(X_embedding.values, dtype=torch.int).to(device)
        self.y = torch.tensor(y.values, dtype=torch.float32).view(-1, 1).to(device)

    def __len__(self):
        return len(self.X_numeric)

    def __getitem__(self, idx):
        return self.X_numeric[idx], self.X_embedding[idx], self.y[idx]


train_dataset = CustomDataset(X_train_numeric, X_train_embedding, y_train)
validation_dataset = CustomDataset(X_validation_numeric, X_validation_embedding, y_validation)
test_dataset = CustomDataset(X_test_numeric, X_test_embedding, y_test)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

mps


In [49]:
import torch.nn as nn
import torch.optim as optim


class RegressionModel(pl.LightningModule):
    def __init__(self, numerical_input_dim, embedding_cardinality, embedding_dim=4):
        super().__init__()
        self.embeddings = nn.ModuleList([
            nn.Embedding(num_categories, embedding_dim)
            for num_categories in embedding_cardinality
        ])
        total_embedding_size = len(embedding_cardinality) * embedding_dim
        self.model = nn.Sequential(
            nn.Linear(numerical_input_dim + total_embedding_size, 128),
            nn.ReLU(),
            nn.Linear(128, 16),
            nn.ReLU(),
            nn.Linear(16, 1)  # Regression output
        )
        self.loss_fn = nn.MSELoss()

    def forward(self, numerical, categorical):
        embedded = [embed_layer(categorical[:, i]) for i, embed_layer in enumerate(self.embeddings)]
        embedded = torch.cat(embedded, dim=1)
        x = torch.cat([embedded, numerical], dim=1)
        return self.model(x)

    def training_step(self, batch, batch_idx):
        numerical, categorical, y = batch
        y_pred = self(numerical, categorical)
        loss = self.loss_fn(y_pred, y)
        self.log("train_loss", loss, prog_bar=True, on_epoch=True, on_step=False)
        return loss

    def validation_step(self, batch, batch_idx):
        numerical, categorical, y = batch
        y_pred = self(numerical, categorical)
        loss = self.loss_fn(y_pred, y)
        self.log("val_loss", loss, prog_bar=True, on_epoch=True, on_step=False)

    def configure_optimizers(self):
        return optim.Adam(self.parameters(), lr=0.01)


model = RegressionModel(numerical_input_dim=X_train_numeric.shape[1],
                        embedding_cardinality=categorical_cardinalities).to(device)
model.train()
checkpoint_callback = ModelCheckpoint(
    monitor="val_loss",
    mode="min",
    save_top_k=1,
    filename="best_model-{epoch:02d}-{val_loss:.4f}",
    verbose=True
)
early_stopping_callback = EarlyStopping(
    monitor="val_loss",
    mode="min",
    patience=5,
    verbose=True
)
trainer = pl.Trainer(max_epochs=50,
                     log_every_n_steps=1,
                     accelerator=str(device),
                     precision="16-mixed",
                     callbacks=[checkpoint_callback, early_stopping_callback])
trainer.fit(model, train_loader, validation_loader)

Using 16bit Automatic Mixed Precision (AMP)
/Users/ozge/PycharmProjects/ozge/kaggle/.env/lib/python3.9/site-packages/torch/amp/grad_scaler.py:132: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn(
GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name       | Type       | Params | Mode 
--------------------------------------------------
0 | embeddings | ModuleList | 804    | train
1 | model      | Sequential | 6.4 K  | train
2 | loss_fn    | MSELoss    | 0      | train
--------------------------------------------------
7.2 K     Trainable params
0         Non-trainable params
7.2 K     Total params
0.029     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/ozge/PycharmProjects/ozge/kaggle/.env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.
/Users/ozge/PycharmProjects/ozge/kaggle/.env/lib/python3.9/site-packages/torch/amp/autocast_mode.py:266: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn(
/Users/ozge/PycharmProjects/ozge/kaggle/.env/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:425: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=11` in the `DataLoader` to improve performance.


Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Epoch 5, global step 24714: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Epoch 6, global step 28833: 'val_loss' was not in top 1


Validation: |          | 0/? [00:00<?, ?it/s]

Monitored metric val_loss did not improve in the last 5 records. Best score: 33114.727. Signaling Trainer to stop.
Epoch 7, global step 32952: 'val_loss' was not in top 1


In [58]:
model = RegressionModel.load_from_checkpoint(checkpoint_callback.best_model_path,
                                             numerical_input_dim=X_train_numeric.shape[1],
                                             embedding_cardinality=categorical_cardinalities).to(device)


In [59]:
model.eval()
test_to_pred_numeric = torch.tensor(X_test_numeric.values, dtype=torch.float32, device=device)
test_to_pred_embedding = torch.tensor(X_test_embedding.values, dtype=torch.int, device=device)
with torch.no_grad():
    predictions = model.to(str(device))(test_to_pred_numeric, test_to_pred_embedding)
predictions = predictions.cpu().numpy().reshape(-1)
print("Predictions:", predictions)

Predictions: [364.94754 208.78683 200.46094 ... 400.96475 383.45883 159.6444 ]


In [60]:
mae = mean_absolute_error(y_test.values, predictions)
mse = mean_squared_error(y_test.values, predictions)
print(f"Mean Absolute Error: {mae}")
print(f"Mean Squared Error: {mse}")

"""
Mean Absolute Error: 94.37028976804564
Mean Squared Error: 32748.60997751085
"""

Mean Absolute Error: 94.37028976804564
Mean Squared Error: 32748.60997751085


'\nMean Absolute Error: 147.2391832704101\nMean Squared Error: 60632.16680838898\n\nMean Absolute Error: 103.89691858055811\nMean Squared Error: 38411.99397804801\n\nMean Absolute Error: 94.41249263131034\nMean Squared Error: 33476.05190155651\n'